In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
data = pd.read_csv('../analyse/operations.csv')
data.head()

,identifiant_transaction,date_operation,date_valeur,libelle,debit,credit,solde
0,242,2023-10-06,2023-10-06,FORFAIT COMPTE SUPERBANK XX XX XX XX,-1.92,NaN,2513.79
1,25,2023-10-06,2023-10-06,CARTE XX XX CHEZ LUC XX,-10.00,NaN,2529.29
2,69,2023-10-06,2023-10-06,PRELEVEMENT XX TELEPHONE XX XX,-13.58,NaN,2515.71
3,299,2023-10-05,2023-10-05,CARTE XX XX XX XX XX XX,-10.64,NaN,2539.29
4,45,2023-10-05,2023-10-05,CARTE XX XX XX XX,-4.80,NaN,2549.93


In [33]:
from collections import Counter

def most_common_words(labels):
    words = []
    for lab in labels:
        words += lab.split(" ")
    counter = Counter(words)
    for word in counter.most_common(100):
        print(word)

In [47]:
#most_common_words(data['libelle'].values)

In [45]:
CATEGS = {
    'LOYER': 'LOYER',
    'FORFAIT COMPTE SUPERBANK': 'COTISATION BANCAIRE',
    'LES ANCIENS ROBINSON': 'COURSES',
    "L'EPICERIE DENBAS": 'COURSES',
    'TELEPHONE': 'FACTURE TELEPHONE',
    'LA CCNCF': 'TRANSPORT',
    'CHEZ LUC': 'RESTAURANT',
    'RAPT': 'TRANSPORT',
    'TOUPTIPRI': 'COURSES',
    'LA LOUVE': 'COURSES',
    'VELOC': 'TRANSPORT'
}
TYPES = {
    'CARTE': 'CARTE',
    'VIR': 'VIREMENT',
    'VIREMENT': 'VIREMENT',
    'RETRAIT': 'RETRAIT',
    'PRLV': 'PRELEVEMENT',
    'DON': 'DON',
}

In [46]:
EXPENSES = [80,200] # Bornes des catégories de dépense : petite, moyenne et grosse
LAST_BALANCE = 2400 # Solde du compte APRES la dernière opération en date
WEEKEND = ["Saturday","Sunday"] # Jours non travaillés

In [92]:
# Controle des colonnes
for c in ['date_operation','libelle','debit','credit']:
    if c not in data.columns:
        if (c in ['debit','credit'] and 'montant' not in data.columns) or \
        (c not in ['debit','credit']):
            msg = "Il vous manque la colonne '{}'. Attention aux majuscules "
            msg += "et minuscules dans le nom des colonnes!"
            raise Exception(msg.format(c))

# Suppression des colonnes innutiles
for c in data.columns:
    if c not in ['date_operation','libelle','debit','credit','montant']:
        del data[c]

# Ajout de la colonne 'montant' si besoin
if 'montant' not in data.columns:
    data["debit"] = data["debit"].fillna(0)
    data["credit"] = data["credit"].fillna(0)
    data["montant"] = data["debit"] + data["credit"]
    del data["credit"], data["debit"]

# creation de la variable 'solde_avt_ope'
data = data.sort_values("date_operation")
amount = data["montant"]
balance = amount.cumsum()
balance = list(balance.values)
last_val = balance[-1]
balance = [0] + balance[:-1]
balance = balance - last_val + LAST_BALANCE
data["solde_avt_ope"] = balance

# Assignation des operations a une categorie et a un type
def detect_words(values, dictionary):
    result = []
    for lib in values:
        operation_type = "AUTRE"
        for word, val in dictionary.items():
            if word in lib:
                operation_type = val
        result.append(operation_type)
    return result
data["categ"] = detect_words(data["libelle"], CATEGS)
data["type"] = detect_words(data["libelle"], TYPES)

# creation des variables 'tranche_depense' et 'sens'
def expense_slice(value):
    value = -value # Les dépenses sont des nombres négatifs
    if value < 0:
        return "(pas une dépense)"
    elif value < EXPENSES[0]:
        return "petite"
    elif value < EXPENSES[1]:
        return "moyenne"
    else:
        return "grosse"
data["tranche_depense"] = data["montant"].map(expense_slice)
data["sens"] = ["credit" if m > 0 else "debit" for m in data["montant"]]

# Creation des autres variables


datetime_object = pd.to_datetime(data["date_operation"])

data["annee"] = datetime_object.map(lambda d: d.year)
data["mois"] = datetime_object.map(lambda d: d.month)
data["jour"] = datetime_object.map(lambda d: d.day)
data["jour_sem"] = datetime_object.map(lambda d: d.day_name)
data["jour_sem_num"] = datetime_object.map(lambda d: d.weekday()+1)
data["weekend"] = data["jour_sem"].isin(WEEKEND)
data["quart_mois"] = [int((jour-1)*4/31)+1 for jour in data["jour"]]
        
# Enregistrement au format CSV
data.to_csv("operations_enrichies.csv",index=False)

In [93]:
new_data = pd.read_csv("operations_enrichies.csv")
new_data.head()

,date_operation,libelle,montant,solde_avt_ope,categ,type,tranche_depense,sens,annee,mois,jour,jour_sem,jour_sem_num,weekend,quart_mois
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE,DON,petite,debit,2023,3,31,<bound method Timestamp.day_name of Timestamp(...,5,False,4
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT,CARTE,petite,debit,2023,4,3,<bound method Timestamp.day_name of Timestamp(...,1,False,1
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT,CARTE,petite,debit,2023,4,3,<bound method Timestamp.day_name of Timestamp(...,1,False,1
3,2023-04-03,CARTE XX XX XX XX,-14.39,1416.81,AUTRE,CARTE,petite,debit,2023,4,3,<bound method Timestamp.day_name of Timestamp(...,1,False,1
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,1402.42,AUTRE,VIREMENT,(pas une dépense),credit,2023,4,3,<bound method Timestamp.day_name of Timestamp(...,1,False,1


In [102]:
effectifs = data["quart_mois"].value_counts()
modalites = effectifs.index # l'index de effectifs contient les modalités

tab = pd.DataFrame(modalites, columns = ["quart_mois"]) # création du tableau à partir des modalités
tab["n"] = effectifs.values
tab["f"] = tab["n"] / len(data) # len(data) renvoie la taille de l'échantillon

tab = tab.sort_values("quart_mois") # tri des valeurs de la variable X (croissant)
tab["F"] = tab["f"].cumsum() # cumsum calcule la somme cumulée

In [103]:
print(tab)

   quart_mois   n         f         F
0           1  86  0.279221  0.279221
1           2  76  0.246753  0.525974
2           3  75  0.243506  0.769481
3           4  71  0.230519  1.000000
